In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Irritable_bowel_syndrome_(IBS)/GSE72221'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Characterization of LncRNA BC012900 among long non-coding RNAs differentially expressed in ulcerative colitis"
!Series_summary	"In this report, we provide a comprehensive assessment of the expression of  ~17000 lncRNAs on 60 colonic samplesin colon tissues from patients with IBD, irritable bowel syndrome, infectious colitis and healthy controls. We also explored the possibility of using cRNAs as biomarkers distinguish active UC from normal.  To investigate the mechanism offunctional role these IBD-associated lncRNAs in the development of IBD, we  then focused on a ncRNA highlyexpressed in the UC-associated lncRNAactive UC, BC012900.  We , to characterized its cellular localization, expression regulation and biological function. We . We found that BC012900 and its adjacent gene, dual specificity phosphatase 4 (DUSP4) are functionally distinct, with BC012900 modulating. Overexpression of BC012900 resulted in usceptibility  to apoptosis. Our study pr

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Checking gene expression data availability
is_gene_available = True

# Variable Availability and Data Type Conversion
# For 'Irritable_bowel_syndrome_(IBS)'
if (len(set(['diagnosis: infectious colitis', 'diagnosis: normal', 'diagnosis: Ulcerative colitis', 'diagnosis: Crohn’s disease', 'diagnosis: irritable bowel syndrome'])) > 1):
    trait_row = 0
    
    def convert_trait(value):
        try:
            trait_value = value.split(':')[1].strip().lower()
            if 'irritable bowel syndrome' in trait_value:
                return 1
            elif any(term in trait_value for term in ['infectious colitis', 'normal', 'ulcerative colitis', 'crohn’s disease']):
                return 0
            else:
                return None
        except:
            return None
else:
    trait_row = None

# 'age' data is not available in the given characteristics, thus remaining as None
age_row = None

# 'gender' data is not available in the given characteristics, thus remaining as None
gender_row = None

# Saving Metadata
save_cohort_info('GSE72221', './preprocessed/Irritable_bowel_syndrome_(IBS)/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Irritable_bowel_syndrome_(IBS)', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Irritable_bowel_syndrome_(IBS)/trait_data/GSE72221.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1857992': [0], 'GSM1857993': [0], 'GSM1857994': [0], 'GSM1857995': [0], 'GSM1857996': [0], 'GSM1857997': [0], 'GSM1857998': [0], 'GSM1857999': [0], 'GSM1858000': [0], 'GSM1858001': [0], 'GSM1858002': [0], 'GSM1858003': [0], 'GSM1858004': [0], 'GSM1858005': [0], 'GSM1858006': [0], 'GSM1858007': [0], 'GSM1858008': [0], 'GSM1858009': [0], 'GSM1858010': [0], 'GSM1858011': [0], 'GSM1858012': [0], 'GSM1858013': [0], 'GSM1858014': [0], 'GSM1858015': [0], 'GSM1858016': [0], 'GSM1858017': [0], 'GSM1858018': [0], 'GSM1858019': [0], 'GSM1858020': [0], 'GSM1858021': [0], 'GSM1858022': [0], 'GSM1858023': [0], 'GSM1858024': [0], 'GSM1858025': [0], 'GSM1858026': [0], 'GSM1858027': [0], 'GSM1858028': [0], 'GSM1858029': [0], 'GSM1858030': [0], 'GSM1858031': [0], 'GSM1858032': [0], 'GSM1858033': [0], 'GSM1858034': [0], 'GSM1858035': [0], 'GSM1858036': [0], 'GSM1858037': [0], 'GSM1858038': [0], 'GSM1858039': [0], 'GSM1858040': [0], 'GSM1858041': [0], 'GSM1858042': [0], 'GSM1858043': [1], 'GSM1858044

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['IVGNh00001', 'IVGNh00002', 'IVGNh00003', 'IVGNh00004', 'IVGNh00005',
       'IVGNh00006', 'IVGNh00007', 'IVGNh00008', 'IVGNh00009', 'IVGNh00010',
       'IVGNh00011', 'IVGNh00012', 'IVGNh00013', 'IVGNh00014', 'IVGNh00015',
       'IVGNh00016', 'IVGNh00017', 'IVGNh00018', 'IVGNh00019', 'IVGNh00020'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['IVGNh10882', 'IVGNh25022', 'IVGNh19914', 'IVGNh35288', 'IVGNh10200'], 'Target_ID': ['AK125605', 'uc001aby', 'BC101837', 'uc003hdp', 'AK124212'], 'Feature': ['ncRNA', 'mRNA', 'mRNA', 'mRNA', 'mRNA'], 'Title': ['CDNA FLJ43617 fis, clone SPLEN2016863', nan, 'Histone cluster 3, H3', nan, 'RAB11 family interacting protein 1 (class I)'], 'Entrez_Gene': [nan, nan, 8290.0, nan, 80223.0], 'coding': [0.0, 1.0, 1.0, 1.0, 1.0], 'noncoding': [1.0, 0.0, 0.0, 0.0, 0.0], 'SEQUENCE': ['ACAGGCGTCCTGTGGGCAGTGCAGAGGTCACCACACTGAGCCAGACTGTCCTCGGCCTTC', 'ACAGCAGCAACTCGGAGGGTGAATGGTCTTGGGATGGAGACCCAGACGCAGAGGCGGGGC', 'AACCTGTGTGTCATCCATGCCAAACGGGTCACCATCATGCCTAAGGACATCCAGCTGGCA', 'TTATGTAACAGGATGGGGCGCTCAAGAATATGCTGGCCACACAGTTCCAGAGCTAAGGCA', 'GACGTATTCTTTGCCATTATTTACAAAGGTGACTCTTTTTTCTTGATATAAAATGTAAGG'], 'Chrom': ['chr16', 'chr1', 'chr1', 'chr4', 'chr8'], 'Start': [892545.0, 870016.0, 226679265.0, 68374162.0, 37841742.0], 'End': [892652.0, 870317.0, 226679325.0, 68375608.0,

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Define identifier_key and gene_symbol_key based on the gene annotation data preview and gene identifiers
identifier_key = 'ID' # Based on the preview, 'ID' stores the same kind of identifiers.
gene_symbol_key = 'Target_ID' # Based on the preview, 'Target_ID' stores the gene symbols.

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library
gene_data = apply_gene_mapping(gene_data, mapping_df)
